# **AI Storytelling**

<img src="assets/logo.jpg" width="500" height="300">

This is Natural Language Processing platform that targets on conversion of short stories to audiobooks with features:

1.	Separated text into dialogues and narrations.
2.	Replaced pronouns by nouns in narrations.
3.	Extracted characters from narration.
4.	Identified genders of characters using Gaussian NB with accuracy of 87%.
5.	Predicted age of characters using Random Forest Regressor with RMSE of 14.6.
6.	Found emotions of characters from dialogues using Transformers Pipeline.
7.	Found pitch, tempo and loudness based on emotions, age and gender.
8.	Performed audio processing for converting dialogues and narrations into audios.
9.	Combined audios to create the complete audiobook. 
10.	Final software development.


Based on the success of the project, it could be extended with image or video processing features in upcoming days.

### One-time Installations

In [ ]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_lg

### Import Libraries and Functions

In [117]:
# basic libraries
import re
import os
import random
import string
import pandas as pd

In [2]:
# nlp libraries
import spacy
nlp = spacy.load("en_core_web_lg")

In [3]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# nlp libraries
import nltk
# nltk.download('wordnet')
# nltk.download("stopwords")
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [5]:
# emotion detection libraries
from transformers import pipeline

In [98]:
# text-to-speech libraries
import pyttsx3
from gtts import gTTS
from moviepy.editor import concatenate_audioclips, AudioFileClip
import librosa
import soundfile as sf

In [7]:
# other libraries
import joblib

### Files and Dataframes Defining

In [8]:
# checking stories
os.listdir("stories")

['beautiful garden.txt',
 'beneath the tides.txt',
 'cafe at midnight.txt',
 'the painted door.txt',
 'threads of imagination.txt']

In [9]:
# input title
title = input("Enter text file name:")

Enter text file name: beautiful garden


In [38]:
# opening text file
story = f"stories/{title}.txt"
with open(story, "r") as f:
    text = f.read()

In [39]:
# create dataframes
df_characters = pd.DataFrame(columns=["cid", "name", "num_dialogue", "num_occurrence"])
df_lines = pd.DataFrame(columns=["pid", "ndid", "character", "dialogue", "narration"])

# 1. Lines Identification

In [40]:
# converting text to paragraphs
paragraphs = text.split("\n")
non_empty_paragraphs = list(filter(lambda x: x != '', paragraphs))

In [41]:
# function to identify narrations and dialogues
def identify_narrations_and_dialogues(paragraph):
    """
    :param paragraph: string of paragraph in a story
    :return: list of tuples in (id, name_of_speaker, dialogue, narration) format
    """
    divisions = paragraph.split('"')
    divisions = list(filter(lambda x: x != '', divisions))
    i, spoken = 0, False
    narrations_and_dialogues = []
    for division in divisions:
        start_index = paragraph.find(division)
        end_index = start_index + len(division) - 1
        try:
            if '"'==paragraph[start_index-1:start_index] and '"'==paragraph[end_index+1:end_index+2] and spoken==False:
                # dialogues
                narrations_and_dialogues.append((i, None, division, None))
                spoken = True
            else:
                # narrations
                narrations_and_dialogues.append((i, None, None, division))
                spoken = False
        except:
            # narrations
            narrations_and_dialogues.append((i, None, None, division))
            spoken = False
        i += 1
    return narrations_and_dialogues


In [42]:
# identifying lines (narrations or dialogues) from each paragraphs
pid_num = 0
for paragraph in non_empty_paragraphs:
    for row in [(pid_num,)+nad for nad in identify_narrations_and_dialogues(paragraph)]:
        df_lines = df_lines._append(pd.Series(row, index=df_lines.columns), ignore_index=True)
    pid_num += 1

In [43]:
# set index using paragraph and nd
df_lines.set_index(['pid', 'ndid'], inplace=True)

In [44]:
# narrations and dialogues
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
1   0         None                                               None   
2   0         None                                               None   
    1         None  You have a heart as beautiful as that butterfl...   
3   0         None                                               None   
    1         None  I've been searching for someone just like you....   
4   0         None                                               None   
    1         None                                  I'd love to help,   
    2         None                                               None   
5   0         None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Once upon a time, in a small, quiet village, t...  
1   0     One sunny afternoon, as Lily was by the river,...  
2   0     Unbeknownst to Lily, a kind stranger had been ...  
    1                                                  None  
3   0     Lily blushed, not used to receiving compliment...  
    1                                                  None  
4   0     Lily's eyes sparkled with excitement. She had ...  
    1                                                  None  
    2                                          she replied.  
5   0     From that day on, Lily spent her afternoons ca...

# 2. Characters Identification

In [45]:
# function to find subjects speaking corresponding dialogues
def identify_subjects(sentence):
    """
    :param sentence: string of sentence in the story
    :return: individual speaker
    """
    tokens = []
    individual, individuals = '', []
    
    for tok in reversed(nlp(sentence)):
        if str(tok)=='.' and tok.dep_ == 'punct':
            break
        tokens.append([tok, tok.dep_])
    tokens = tokens[::-1]
    
    for i in range(len(tokens)-1, -1, -1):
        if tokens[i][1] == "nsubj":
            individual = str(tokens[i][0])
        elif (tokens[i][1] in ['compound', 'det'] and individual != ''):
            individual = str(tokens[i][0]) + " " + individual
        else:
            individuals.append(individual)
            
    individuals.append(individual)
    return list(set([ind for ind in individuals if ind != '']))

In [46]:
# finding semantic similarity
def calculating_semantic_similarity(subject, sentence):
    """
    :param subject: list of strings available in a sentence
    :return: similarity score
    """
    subject_doc = nlp(subject)
    sentence_doc = nlp(sentence)
    
    subject_vec = subject_doc.vector
    sentence_vec = sentence_doc.vector
    
    subject_vec_norm = subject_doc.vector_norm
    sentence_vec_norm = sentence_doc.vector_norm
    
    if subject_vec_norm > 0 and sentence_vec_norm > 0:
        similarity_score = subject_vec.dot(sentence_vec) / (subject_vec_norm * sentence_vec_norm)
    else:
        similarity_score = 0
    
    return similarity_score

In [47]:
# finding subjects from dialogues and narrations
dlg_list = df_lines.index[df_lines['dialogue'].notna()].tolist()

for dlgi in dlg_list:
    if df_lines.loc[(dlgi[0], dlgi[1]), 'dialogue'][-1] == ',':
        full_narration = df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration']
        final_narration = full_narration.rstrip().rstrip(string.punctuation)
        subjects = identify_subjects(final_narration)
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k]) if relevant_dictionary else None
    else:
        full_narration = df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration']
        final_narration = full_narration.rstrip().rstrip(string.punctuation)
        subjects = identify_subjects(final_narration)
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k]) if relevant_dictionary else None
    df_lines.loc[(dlgi[0], dlgi[1]), 'character'] = relevant_subject

In [48]:
# dialogues and corresponding characters
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                     
0   0              None                                               None   
1   0              None                                               None   
2   0              None                                               None   
    1      the stranger  You have a heart as beautiful as that butterfl...   
3   0              None                                               None   
    1     mrs. thompson  I've been searching for someone just like you....   
4   0              None                                               None   
    1               she                                  I'd love to help,   
    2              None                                               None   
5   0              None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Once upon a time, in a small, quiet village, t...  
1   0     One sunny afternoon, as Lily was by the river,...  
2   0     Unbeknownst to Lily, a kind stranger had been ...  
    1                                                  None  
3   0     Lily blushed, not used to receiving compliment...  
    1                                                  None  
4   0     Lily's eyes sparkled with excitement. She had ...  
    1                                                  None  
    2                                          she replied.  
5   0     From that day on, Lily spent her afternoons ca...

In [49]:
# check characters
df_lines.character.unique()

array([None, 'the stranger', 'mrs. thompson', 'she', 'that'], dtype=object)

In [50]:
# gathering characters from df_lines
characters = df_lines['character'].value_counts()
df_characters['name'] = characters.index
df_characters['num_dialogue'] = characters.values

In [51]:
# declaring cid as index
df_characters['cid'] = range(0, len(df_characters))
df_characters.set_index('cid', inplace=True)

In [52]:
# counting number of times these names occurred in text
df_characters['num_occurrence'] = df_characters['name'].apply(lambda x: len(re.findall(x, text.lower())))

In [53]:
# top 10 characters
df_characters.head(10)

,name,num_dialogue,num_occurrence
cid,,,
0,the stranger,1,1
1,mrs. thompson,1,8
2,she,1,11
3,that,1,6


# 3. Cleaned Dialogue Generation

In [54]:
# defining stopwords set and lemma
set_of_stopwords = set(stopwords.words("english"))
lemma = WordNetLemmatizer()

In [55]:
# function to remove contractions and replace by spaces
def remove_contractions(text):

    contractions = ["ain't", "aren't", "can't", "can't've", "'cause", "could've", "couldn't", "couldn't've", "didn't", "doesn't", "don't",
                    "hadn't", "hadn't've", "hasn't", "haven't", "he'd", "he'd've", "he'll", "he'll've", "he's", "how'd", "how'd'y", "how'll", "how's",
                    "I'd", "I'd've", "I'll", "I'll've", "I'm", "I've", "isn't", "it'd", "it'd've", "it'll", "it'll've", "it's", "let's",
                    "ma'am", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've",
                    "o'clock", "oughtn't", "oughtn't've", "shan't", "sha'n't", "shan't've", "she'd", "she'd've", "she'll", "she'll've", "she's",
                    "should've", "shouldn't", "shouldn't've", "so've", "so's", "that'd", "that'd've", "that's", "there'd", "there'd've", "there's",
                    "they'd", "they'd've", "they'll", "they'll've", "they're", "they've", "to've", "wasn't",
                    "we'd", "we'd've", "we'll", "we'll've", "we're", "we've", "weren't", "what'll", "what'll've", "what're", "what's", "what've",
                    "when's", "when've", "where'd", "where's", "where've", "who'll", "who'll've", "who's", "who've", "why's", "why've",
                    "will've", "won't", "won't've", "would've", "wouldn't", "wouldn't've", "y'all", "y'all'd", "y'all'd've", "y'all're", "y'all've",
                    "you'd", "you'd've", "you'll", "you'll've", "you're", "you've", "gonna"]

    for contraction in contractions:
        text = text.replace(contraction, " ")

    return text

In [56]:
# function to remove punctuations
def remove_punctuations(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [57]:
# function to check noun
def is_noun(word):
    synsets = wordnet.synsets(word)
    for synset in synsets:
        if synset.pos() == 'n':
            return True
    return False

In [58]:
# dialogue to cleaned dialogue
def cleaned_dialogue(text):
    if text is None:
        return ''
    contractionless_text = remove_contractions(text)
    punctuationless_text = remove_punctuations(contractionless_text)
    tokens = nltk.word_tokenize(punctuationless_text)
    filtered_words = [token.lower() for token in tokens if token.lower() not in set_of_stopwords]
    lemmatized_words = [lemma.lemmatize(word) for word in filtered_words]
    words = [word for word in lemmatized_words if is_noun(word)]
    return ' '.join(words)

In [59]:
# list of words for dialogue
dialogue_list = []
for dialogue in df_lines.dialogue:
    dialogue_list.append(cleaned_dialogue(dialogue))
df_lines['cleaned_dialogue'] = dialogue_list

In [60]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                     
0   0              None                                               None   
1   0              None                                               None   
2   0              None                                               None   
    1      the stranger  You have a heart as beautiful as that butterfl...   
3   0              None                                               None   
    1     mrs. thompson  I've been searching for someone just like you....   
4   0              None                                               None   
    1               she                                  I'd love to help,   
    2              None                                               None   
5   0              None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Once upon a time, in a small, quiet village, t...   
1   0     One sunny afternoon, as Lily was by the river,...   
2   0     Unbeknownst to Lily, a kind stranger had been ...   
    1                                                  None   
3   0     Lily blushed, not used to receiving compliment...   
    1                                                  None   
4   0     Lily's eyes sparkled with excitement. She had ...   
    1                                                  None   
    2                                          she replied.   
5   0     From that day on, Lily spent her afternoons ca...   

                                           cleaned_dialogue  
pid ndid                                                     
0   0                                                        
1   0                                                        
2   0                                                        
    1                                  heart butterfly dear  
3   0                                                        
    1     someone like garden full flower need help tending  
4   0                                                        
    1                                             love help  
    2                                                        
5   0

# 4. Age Detection

In [61]:
# loading age count vectorizer
loaded_age_cv = joblib.load('attributes/age/age_cv.joblib')

In [62]:
# creating sparse matrix for transformed values and array
age_sparse_matrix = loaded_age_cv.transform(df_lines['cleaned_dialogue'])
age_sparse_matrix_array = age_sparse_matrix.toarray()

In [63]:
# loading age model
loaded_age_model = joblib.load('attributes/age/age_model.joblib')

In [64]:
# defining X and predicting
df_lines['age'] = loaded_age_model.predict(age_sparse_matrix_array)

In [65]:
# define age as None if dialogue is None
df_lines['age'] = df_lines.apply(lambda row: None if pd.isna(row['dialogue']) else row['age'], axis=1)

In [66]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                     
0   0              None                                               None   
1   0              None                                               None   
2   0              None                                               None   
    1      the stranger  You have a heart as beautiful as that butterfl...   
3   0              None                                               None   
    1     mrs. thompson  I've been searching for someone just like you....   
4   0              None                                               None   
    1               she                                  I'd love to help,   
    2              None                                               None   
5   0              None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Once upon a time, in a small, quiet village, t...   
1   0     One sunny afternoon, as Lily was by the river,...   
2   0     Unbeknownst to Lily, a kind stranger had been ...   
    1                                                  None   
3   0     Lily blushed, not used to receiving compliment...   
    1                                                  None   
4   0     Lily's eyes sparkled with excitement. She had ...   
    1                                                  None   
    2                                          she replied.   
5   0     From that day on, Lily spent her afternoons ca...   

                                           cleaned_dialogue   age  
pid ndid                                                           
0   0                                                         NaN  
1   0                                                         NaN  
2   0                                                         NaN  
    1                                  heart butterfly dear  5.02  
3   0                                                         NaN  
    1     someone like garden full flower need help tending  6.09  
4   0                                                         NaN  
    1                                             love help  5.27  
    2                                                         NaN  
5   0                                                         NaN

In [67]:
# calculating age of character
for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'age'] = df_lines[df_lines['character'] == character]['age'].mean()

In [68]:
# top 10 rows with age
df_characters.head(10)

,name,num_dialogue,num_occurrence,age
cid,,,,
0,the stranger,1,1,5.02
1,mrs. thompson,1,8,6.09
2,she,1,11,5.27
3,that,1,6,24.66


# 5. Gender Detection

In [69]:
# loading gender count vectorizer
loaded_gender_cv = joblib.load('attributes/gender/gender_cv.joblib')

In [70]:
# creating sparse matrix for transformed values and array
gender_sparse_matrix = loaded_gender_cv.transform(df_lines['cleaned_dialogue'])
gender_sparse_matrix_array = gender_sparse_matrix.toarray()

In [71]:
# loading gender model
loaded_gender_model = joblib.load('attributes/gender/gender_model.joblib')

In [72]:
# defining X and predicting
df_lines['gender'] = loaded_gender_model.predict(gender_sparse_matrix_array)

In [73]:
# define gender as None if dialogue is None
df_lines.loc[df_lines['dialogue'].isna(), 'gender'] = None

In [74]:
# define gender based upon he/she
df_lines['gender'] = df_lines.apply(lambda row: 'Male' if row['character']=='he' else 'Female' if row['character']=='she' else row['gender'], axis=1)

In [75]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                     
0   0              None                                               None   
1   0              None                                               None   
2   0              None                                               None   
    1      the stranger  You have a heart as beautiful as that butterfl...   
3   0              None                                               None   
    1     mrs. thompson  I've been searching for someone just like you....   
4   0              None                                               None   
    1               she                                  I'd love to help,   
    2              None                                               None   
5   0              None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Once upon a time, in a small, quiet village, t...   
1   0     One sunny afternoon, as Lily was by the river,...   
2   0     Unbeknownst to Lily, a kind stranger had been ...   
    1                                                  None   
3   0     Lily blushed, not used to receiving compliment...   
    1                                                  None   
4   0     Lily's eyes sparkled with excitement. She had ...   
    1                                                  None   
    2                                          she replied.   
5   0     From that day on, Lily spent her afternoons ca...   

                                           cleaned_dialogue   age  gender  
pid ndid                                                                   
0   0                                                         NaN    None  
1   0                                                         NaN    None  
2   0                                                         NaN    None  
    1                                  heart butterfly dear  5.02  Female  
3   0                                                         NaN    None  
    1     someone like garden full flower need help tending  6.09  Female  
4   0                                                         NaN    None  
    1                                             love help  5.27  Female  
    2                                                         NaN    None  
5   0                                                         NaN    None

In [76]:
# defining empty columns
df_characters['male_count'], df_characters['female_count'] = None, None

In [77]:
# counting male and female values of character
for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'male_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Male")])
    df_characters.loc[df_characters['name'] == character, 'female_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Female")])

In [78]:
# defining gender column
df_characters['gender'] = df_characters.apply(lambda row: 'Female' if row['female_count'] > row['male_count'] else ('Male' if row['female_count'] < row['male_count'] else None), axis=1)

In [79]:
# top 10 rows with male and female counts
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,the stranger,1,1,5.02,0,1,Female
1,mrs. thompson,1,8,6.09,0,1,Female
2,she,1,11,5.27,0,1,Female
3,that,1,6,24.66,1,0,Male


# 6. He/She Replacement

In [80]:
# characters not pronouns
other_characters = df_characters[~df_characters['name'].isin(['he', 'she', 'that', 'they', 'it'])]['name']
other_characters = list(other_characters)
print(other_characters)

['the stranger', 'mrs. thompson']


In [81]:
# names in narration
def find_names_in_narration(narration):
    if narration is None:
        return None
    names = dict()
    for character in other_characters:
        location = narration.lower().rfind(character)
        if location != -1:
            names[character] = location
    if len(names) == 0:
        return None
    return names.keys()
df_lines['narrated_names'] = df_lines['narration'].apply(lambda row: find_names_in_narration(row))

In [82]:
# he she replacement by names
def he_she_replacement(i, gender):
    j = i-1
    last_row_narrated_names = df_lines.iloc[j]['narrated_names']
    while last_row_narrated_names is None:
        j -= 1
        last_row_narrated_names = df_lines.iloc[j]['narrated_names']
    last_row_narrated_names = list(last_row_narrated_names)
    for name in last_row_narrated_names:
        if list(df_characters[df_characters['name'] == name]['gender'])[0] == gender:
            return name
    return last_row_narrated_names[0]

In [83]:
# replacing all he/she in df_lines
for i in range(1, len(df_lines)):
    if df_lines.iloc[i, 0] not in other_characters+['he', 'she'] and df_lines.iloc[i, 0] is not None:
        df_lines.iloc[i, 0] = None
        df_lines.iloc[i, 4] = None
    elif df_lines.iloc[i, 0] == 'he':
        df_lines.iloc[i, 0] = he_she_replacement(i, 'Male')
    elif df_lines.iloc[i, 0] == 'she':
        df_lines.iloc[i, 0] = he_she_replacement(i, 'Female')
    else:
        pass

In [84]:
# lines with narrated names
df_lines.tail(10)

character                                           dialogue  \
pid ndid                                                                     
4   0              None                                               None   
    1     mrs. thompson                                  I'd love to help,   
    2              None                                               None   
5   0              None                                               None   
6   0              None                                               None   
7   0              None                                               None   
8   0              None                                               None   
    1              None  May the kindness you showed to a struggling bu...   
9   0              None                                               None   
10  0              None                                               None   

                                                  narration  \
pid ndid                                                      
4   0     Lily's eyes sparkled with excitement. She had ...   
    1                                                  None   
    2                                          she replied.   
5   0     From that day on, Lily spent her afternoons ca...   
6   0     Lily and Mrs. Thompson developed a deep and he...   
7   0     As the years went by, Lily continued to visit ...   
8   0     One fateful day, Mrs. Thompson passed away, le...   
    1                                                  None   
9   0     Lily kept her promise, and the garden continue...   
10  0     And so, in the memory of a chance encounter an...   

                                    cleaned_dialogue   age  gender  \
pid ndid                                                             
4   0                                                  NaN    None   
    1                                      love help  5.27  Female   
    2                                                  NaN    None   
5   0                                                  NaN    None   
6   0                                                  NaN    None   
7   0                                                  NaN    None   
8   0                                                  NaN    None   
    1     may kindness butterfly world beauty flower   NaN    Male   
9   0                                                  NaN    None   
10  0                                                  NaN    None   

           narrated_names  
pid ndid                   
4   0     (mrs. thompson)  
    1                None  
    2                None  
5   0     (mrs. thompson)  
6   0     (mrs. thompson)  
7   0     (mrs. thompson)  
8   0     (mrs. thompson)  
    1                None  
9   0     (mrs. thompson)  
10  0                None

In [85]:
# filtered df_characters with other_characters
df_characters = df_characters[df_characters['name'].isin(other_characters)]

In [86]:
# filtered df_characters only
df_characters.head()

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,the stranger,1,1,5.02,0,1,Female
1,mrs. thompson,1,8,6.09,0,1,Female


In [87]:
# gathering characters from df_lines
df_lines_vc = df_lines['character'].value_counts()
filtered_df = pd.DataFrame(df_lines_vc.reset_index(), columns=['character', 'count'])

In [88]:
# filtered_df
filtered_df.head()

,character,count
0,mrs. thompson,2
1,the stranger,1


In [89]:
# number of dialogues, male and female counts and gender after he/she replacement
df_characters['num_dialogue'] = df_characters['name'].apply(lambda x: list(filtered_df[filtered_df['character'] == x]['count'])[0])

for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'male_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Male")])
    df_characters.loc[df_characters['name'] == character, 'female_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Female")])

df_characters['gender'] = df_characters.apply(lambda row: 'Female' if row['female_count'] > row['male_count'] else ('Male' if row['female_count'] < row['male_count'] else None), axis=1)

In [90]:
# new df_characters with new number of dialogues, male count and female count
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,the stranger,1,1,5.02,0,1,Female
1,mrs. thompson,2,8,6.09,0,2,Female


# 7. Age and Gender Editing

In [92]:
# inputs for new age and gender
for index, row in df_characters.iterrows():
    print(row['name'].upper())
    print(f"Age predicted:{row['age']}")
    new_age = input("Enter age (or leave blank if you are ok with predicted age.)")
    df_characters.loc[df_characters['name'] == row['name'], 'age'] = float(new_age) if new_age != '' else row['age']
    print(f"Gender predicted:{row['gender']}")
    new_gender = input("Enter gender (Male/Female/None or leave blank if you are ok with predicted gender.)")
    df_characters.loc[df_characters['name'] == row['name'], 'gender'] = new_gender if new_gender != '' else row['gender']
    print("======================================")

THE STRANGER
Age predicted:40.0


Enter age (or leave blank if you are ok with predicted age.) 40


Gender predicted:Female


Enter gender (Male/Female/None or leave blank if you are ok with predicted gender.) Female


MRS. THOMPSON
Age predicted:40.0


Enter age (or leave blank if you are ok with predicted age.) 40


Gender predicted:Female


Enter gender (Male/Female/None or leave blank if you are ok with predicted gender.) Female


In [93]:
# replace 'None' by None
df_characters[df_characters['gender'] == 'None'] = None

In [94]:
# checking ages
df_characters['age'].unique()

array([40.])

In [95]:
# checking genders
df_characters['gender'].unique()

array(['Female'], dtype=object)

In [96]:
# checking characters
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,the stranger,1,1,40.0,0,1,Female
1,mrs. thompson,2,8,40.0,0,2,Female


# 8. Gender-based Voice Selection

In [100]:
# create voice engine for voice selection
class VoiceEngine:

    def __init__(self, character):
        self.engine = pyttsx3.init()
        voices = self.engine.getProperty('voices')
        speakers = {
            'David': voices[0],
            'James': voices[1],
            'Linda': voices[2],
            'Richard': voices[3],
            'George': voices[4],
            'Susan': voices[5],
            'Sean': voices[6],
            'Heera': voices[7],
            'Ravi': voices[8],
            'Mark': voices[9],
            'Hazel': voices[10],
            'Catherine': voices[11],
            'Zira': voices[12]
        }
        self.engine.setProperty('voice', speakers[character].id)

    def dialogue_delivery(self, dialogue):
        self.engine.say(dialogue)
        self.engine.runAndWait()

In [116]:
# definining characters
males = ['David', 'James', 'Richard', 'George', 'Sean', 'Ravi', 'Mark']
females = ['Linda', 'Susan', 'Heera', 'Hazel', 'Catherine', 'Zira']

# unused characters
unused_males = males.copy()
unused_females = females.copy()

In [115]:
# empty column
df_characters['voice'] = None

In [118]:
# allocating voice to all characters
for index, row in df_characters.iterrows():
    if row['gender'] == 'Male':
        random_number = random.randint(0, len(unused_males)-1)
        name_of_voice = unused_males[random_number]
        df_characters.loc[index, 'voice'] = name_of_voice
        unused_males.remove(name_of_voice)
        if len(unused_males) == 0:
            unused_males = males.copy()
    elif row['gender'] == 'Female':
        random_number = random.randint(0, len(unused_females)-1)
        name_of_voice = unused_females[random_number]
        df_characters.loc[index, 'voice'] = name_of_voice
        unused_females.remove(name_of_voice)
        if len(unused_females) == 0:
            unused_females = females.copy()
    else:
        unused_all = unused_males + unused_females
        random_number = random.randint(0, len(unused_all)-1)
        name_of_voice = unused_all[random_number]
        df_characters.loc[index, 'voice'] = name_of_voice
        if name_of_voice in unused_males:
            unused_males.remove(name_of_voice)
            if len(unused_males) == 0:
                unused_males = males.copy()
        else:
            unused_females.remove(name_of_voice)
            if len(unused_females) == 0:
                unused_females = females.copy()

In [119]:
# checking characters with voice
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender,voice
cid,,,,,,,,
0,the stranger,1,1,40.0,0,1,Female,Linda
1,mrs. thompson,2,8,40.0,0,2,Female,Heera


In [138]:
# checking remaining voices
print(unused_males, unused_females)

['David', 'James', 'Richard', 'George', 'Sean', 'Ravi', 'Mark'] ['Susan', 'Hazel', 'Catherine', 'Zira']


In [182]:
# assigning voice to narrator
unused_all = unused_males + unused_females
random_number = random.randint(0, len(unused_all)-1)
narrator_voice = unused_all[random_number]
print(f"Narrator voice: {narrator_voice}")

Narrator voice: Zira


# 8. Pitch-Tempo Defining by Age

In [135]:
# pitch estimation in Hz and tempo estimation in words per minute by age
def pitch_and_tempo_estimation(age):
    if age <= 5:
        result = [300, 150]
    elif age > 5 and age <= 12:
        result = [250, 160]
    elif age > 12 and age <= 18:
        result = [200, 170]
    elif age > 18 and age <= 30:
        result = [180, 180]
    elif age > 30 and age <= 40:
        result = [160, 175]
    elif age > 40 and age <= 60:
        result = [150, 160]
    else:
        result = [130, 150]
    return pd.Series(result, index=('pitch', 'tempo'))

In [136]:
# adding pitch and tempo columns by characters' age
df_characters[['pitch', 'tempo']] = df_characters['age'].apply(pitch_and_tempo_estimation)

In [137]:
# checking table with pitch, tempo and voice
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender,voice,pitch,tempo
cid,,,,,,,,,,
0,the stranger,1,1,40.0,0,1,Female,Linda,160,175
1,mrs. thompson,2,8,40.0,0,2,Female,Heera,160,175


# 9. Emotions Detection

In [90]:
# creating emotion detection classifier
ed_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [91]:
# function to return better dictionary
def show_emotions(dialogue):
    emotions_dict = dict()
    edc = ed_classifier(dialogue)
    for x in edc[0]:
        emotions_dict[x['label']] = x['score']
    return emotions_dict

In [92]:
# using ed_classifier for showing emotions for each dialogue
df_lines['anger'], df_lines['disgust'], df_lines['fear'], df_lines['joy'], df_lines['neutral'], df_lines['sadness'], df_lines['surprise'] = None, None, None, None, None, None, None
df_lines[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']] = df_lines['dialogue'].apply(lambda x: list(show_emotions(x).values()) if x is not None else [None, None, None, None, None, None, None]).tolist()

In [93]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                    
0   0             None                                               None   
1   0             None                                               None   
2   0             None                                               None   
    1             alex                 Late night for a coffee, isn't it?   
3   0             None                                               None   
    1     the stranger         Some stories unfold when the world sleeps.   
4   0             None                                               None   
    1             alex                              What kind of stories?   
5   0     the stranger  Stories of the forgotten, the ones that only e...   
    1             None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Midnight in the heart of the city, the neon si...   
1   0     The door creaked open, and a mysterious figure...   
2   0                              Alex raised an eyebrow.    
    1                                                  None   
3   0     The stranger chuckled, a voice tinged with int...   
    1                                                  None   
4   0                      Intrigued, Alex leaned forward.    
    1                                                  None   
5   0                                                  None   
    1                    the stranger replied mysteriously.   

                cleaned_dialogue    age  gender  narrated_names     anger  \
pid ndid                                                                    
0   0                               NaN    None          (alex)       NaN   
1   0                               NaN    None          (alex)       NaN   
2   0                               NaN    None          (alex)       NaN   
    1               night coffee  20.47  Female            None  0.061734   
3   0                               NaN    None  (the stranger)       NaN   
    1          story world sleep  14.27    Male            None  0.015564   
4   0                               NaN    None          (alex)       NaN   
    1                 kind story   5.31    Male            None  0.015280   
5   0     story one city silence   5.31    Male            None  0.010006   
    1                               NaN    None  (the stranger)       NaN   

           disgust      fear       joy   neutral   sadness  surprise  
pid ndid                                                              
0   0          NaN       NaN       NaN       NaN       NaN       NaN  
1   0          NaN       NaN       NaN       NaN       NaN       NaN  
2   0          NaN       NaN       NaN       NaN       NaN       NaN  
    1     0.377704  0.022014  0.010213  0.325844  0.041864  0.160626  
3   0          NaN       NaN       NaN       NaN       NaN       NaN  
    1     0.095981  0.034734  0.003982  0.776333  0.044221  0.029184  
4   0          NaN       NaN       NaN       NaN       NaN       NaN  
    1     0.010232  0.004569  0.004578  0.831131  0.003179  0.131032  
5   0     0.046537  0.044176  0.002887  0.560465  0.322675  0.013254  
    1          NaN       NaN       NaN       NaN       NaN       NaN

In [94]:
# check dialogues and corresponding emotions only
df_lines[~df_lines['dialogue'].isna()][['dialogue', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']].head(10)

dialogue     anger  \
pid ndid                                                                
2   1                    Late night for a coffee, isn't it?  0.061734   
3   1            Some stories unfold when the world sleeps.  0.015564   
4   1                                 What kind of stories?  0.015280   
5   0     Stories of the forgotten, the ones that only e...  0.010006   
7   0                                             You know,  0.076726   
    2     this feels like a chapter from a novel. A meet...  0.011276   
8   1     Perhaps it is. Life has a way of scripting its...  0.005774   
9   1                    Until our paths cross again, Alex.  0.015226   

           disgust      fear       joy   neutral   sadness  surprise  
pid ndid                                                              
2   1     0.377704  0.022014  0.010213  0.325844  0.041864  0.160626  
3   1     0.095981  0.034734  0.003982  0.776333  0.044221  0.029184  
4   1     0.010232  0.004569  0.004578  0.831131  0.003179  0.131032  
5   0     0.046537  0.044176  0.002887  0.560465  0.322675  0.013254  
7   0     0.240470  0.025607  0.024524  0.607015  0.013988  0.011670  
    2     0.111364  0.222619  0.005216  0.544812  0.012213  0.092499  
8   1     0.010377  0.002170  0.002140  0.962292  0.004725  0.012522  
9   1     0.018030  0.068527  0.014377  0.815693  0.064210  0.003937

# 10. Audio Generation

In [102]:
# creating audios
print("Step 1: CONVERSIONS")
i = 0
for index, row in df_lines.iterrows():
    if row['dialogue'] is not None:
        speech_gtts = gTTS(text=row['dialogue'], lang='en', slow=False, tld='co.in')
    else:
        speech_gtts = gTTS(text=row['narration'], lang='en', slow=False, tld='us')
    temp_file = f"conversions/{i}.mp3"
    print(f"Line {i+1}/{df_lines.shape[0]} converted.")
    speech_gtts.save(temp_file)
    i += 1
print("Conversions finished.")

Step 1: CONVERSIONS
Line 1/19 converted.
Line 2/19 converted.
Line 3/19 converted.
Line 4/19 converted.
Line 5/19 converted.
Line 6/19 converted.
Line 7/19 converted.
Line 8/19 converted.
Line 9/19 converted.
Line 10/19 converted.
Line 11/19 converted.
Line 12/19 converted.
Line 13/19 converted.
Line 14/19 converted.
Line 15/19 converted.
Line 16/19 converted.
Line 17/19 converted.
Line 18/19 converted.
Line 19/19 converted.
Conversions finished.


In [103]:
# combining audios
print("Step 2: COMBINATION")
clips = [AudioFileClip(f"conversions/{i}.mp3") for i in range(df_lines.shape[0])]
final_clip = concatenate_audioclips(clips)
print("Combination finished.")

Step 2: COMBINATION
Combination finished.


In [104]:
# adjust speed
print("Step 3: ADJUSTMENT")
final_clip.write_audiofile("conversions/final_slow.mp3")
print("final_slow.mp3 downloaded.")
y, sr = librosa.load("conversions/final_slow.mp3", sr=None)
y_speed = librosa.effects.time_stretch(y, rate=1.25)
print("Speeded over.")

Step 3: ADJUSTMENT
MoviePy - Writing audio in conversions/final_slow.mp3


MoviePy - Done.
final_slow.mp3 downloaded.
Speeded over.


In [105]:
# removing audios
print("Step 4: REMOVAL")
for i in range(df_lines.shape[0]):
    os.remove(f"conversions/{i}.mp3")
    print(f"Removed {i}.mp3")
os.remove("conversions/final_slow.mp3")
print("Removal over.")

Step 4: REMOVAL
Removed 0.mp3
Removed 1.mp3
Removed 2.mp3
Removed 3.mp3
Removed 4.mp3
Removed 5.mp3
Removed 6.mp3
Removed 7.mp3
Removed 8.mp3
Removed 9.mp3
Removed 10.mp3
Removed 11.mp3
Removed 12.mp3
Removed 13.mp3
Removed 14.mp3
Removed 15.mp3
Removed 16.mp3
Removed 17.mp3
Removed 18.mp3
Removal over.


In [106]:
# downloading final audio
print("Step 5: DOWNLOADING")
final_title = title.replace(" ", "_")
sf.write(f"audiobooks/{final_title}.mp3", y_speed, sr)
print(f"{final_title}.mp3 downloaded")

Step 5: DOWNLOADING
cafe_at_midnight.mp3 downloaded
